In [1]:
import pandas as pd 


sp600 = pd.read_csv('index/sp600_small_cap.csv')
sp600.head()

,Symbol,Company,GICS Sector,GICS Sub-Industry,Headquarters Location,CIK
0,AAP,"Advance Auto Parts, Inc.",Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",1158449
1,AAT,American Assets Trust,Real Estate,Diversified REITs,"San Diego, California",1500217
2,ABCB,Ameris Bancorp,Financials,Regional Banks,"Atlanta, Georgia",351569
3,ABG,Asbury Automotive Group,Consumer Discretionary,Automotive Retail,"Duluth, Georgia",1144980
4,ABM,"ABM Industries, Inc.",Industrials,Environmental & Facilities Services,"New York City, New York",771497


In [2]:
assert sp600['Symbol'].notna().all()

In [3]:
symbols = sp600['Symbol'].to_list()

In [4]:

from src.utils import download_max

df = download_max(symbols=symbols)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  602 of 602 completed

8 Failed downloads:
['CWEN.A']: HTTPError('HTTP Error 404: ')
['BSIG', 'UCBI', 'JBT', 'DRQ', 'MOG.A', 'VVI', 'SGH']: YFTzMissingError('possibly delisted; no timezone found')


In [5]:
failed_symbols = ['BSIG', 'UCBI', 'JBT', 'DRQ', 'MOG.A', 'VVI', 'SGH', 'CWEN.A']

In [7]:
sp600_errors = sp600[sp600['Symbol'].isin(set(failed_symbols))]
sp600_errors

,Symbol,Company,GICS Sector,GICS Sub-Industry,Headquarters Location,CIK
91,BSIG,"BrightSphere Investment Group, Inc.",Financials,Asset Management & Custody Banks,"Boston, Massachusetts",1748824
151,CWEN.A,"Clearway Energy, Inc. (Class A)",Utilities,Renewable Electricity,"Princeton, New Jersey",1567683
169,DRQ,Dril-Quip Inc.,Energy,Oil & Gas Equipment & Services,"Houston, Texas",1042893
288,JBT,John Bean Technologies Corporation,Industrials,Industrial Machinery,"Chicago, Illinois",1433660
345,MOG.A,Moog Inc.,Industrials,Aerospace & Defense,"Elma, New York",67887
473,SGH,"SMART Global Holdings, Inc.",Information Technology,Semiconductors,"Newark, California",1616533
543,UCBI,"United Community Banks, Inc.",Financials,Regional Banks,"Greenville, South Carolina",857855
574,VVI,Viad Corp,Industrials,Diversified Support Services,"Scottsdale, Arizona",884219


In [10]:
sp600_vaild = sp600[~sp600['Symbol'].isin(set(failed_symbols))]
sp600_vaild.head()

,Symbol,Company,GICS Sector,GICS Sub-Industry,Headquarters Location,CIK
0,AAP,"Advance Auto Parts, Inc.",Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",1158449
1,AAT,American Assets Trust,Real Estate,Diversified REITs,"San Diego, California",1500217
2,ABCB,Ameris Bancorp,Financials,Regional Banks,"Atlanta, Georgia",351569
3,ABG,Asbury Automotive Group,Consumer Discretionary,Automotive Retail,"Duluth, Georgia",1144980
4,ABM,"ABM Industries, Inc.",Industrials,Environmental & Facilities Services,"New York City, New York",771497


In [11]:
assert sp600.shape[0] - sp600_vaild.shape[0] == 8

In [12]:
symbols_valid = sp600_vaild['Symbol'].to_list()

In [14]:
sp600_vaild.to_csv('index/sp600_small_cap_corrected.csv')

In [13]:
from src.utils import download_max

df = download_max(symbols=symbols_valid)

[*********************100%***********************]  594 of 594 completed
/home/ping/Code/proto-collect-stock-data/src/utils.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume'] = df['Volume'].fillna(0).astype(int)


In [15]:
df.head()

,trade_date,ticker,open_price,high_price,low_price,close_price,volume
0,1962-01-02,XRX,0.0,0.6618,0.6536,0.6577,51233
1,1962-01-02,FL,0.0,0.8800,0.8700,0.8700,49200
2,1962-01-03,XRX,0.0,0.6700,0.6618,0.6700,51233
3,1962-01-03,FL,0.0,0.8600,0.8600,0.8600,31200
4,1962-01-04,XRX,0.0,0.6834,0.6659,0.6659,198099


In [16]:
df.tail()

,trade_date,ticker,open_price,high_price,low_price,close_price,volume
3737261,2025-05-30,JXN,81.87,82.44,80.82,81.91,950000
3737262,2025-05-30,NEOG,6.16,6.25,5.85,5.86,3697800
3737263,2025-05-30,RHI,46.14,46.21,45.03,45.79,1110100
3737264,2025-05-30,DVAX,9.72,9.87,9.59,9.79,1937300
3737265,2025-05-30,GDYN,12.61,12.90,12.48,12.53,923200


In [17]:
from src.utils import get_db_engine

engine = get_db_engine()

In [18]:
from importlib import reload
from src import utils
reload(utils)

from src.utils import get_latest_record_date

latest_df = get_latest_record_date(engine=engine)
latest_df.head()

,ticker,latest_record_date
0,A,2025-05-08
1,AA,2025-05-08
2,AAL,2025-05-08
3,AAON,2025-05-08
4,AAPL,2025-05-08


check if all tickers share the same latest record date

In [19]:
assert (latest_df['latest_record_date'] == latest_df['latest_record_date'].max()).all()

In [20]:
date_latest_on_db = latest_df['latest_record_date'].max()
date_latest_on_db

datetime.date(2025, 5, 8)

In [21]:
df_trimmed = df[df['trade_date'] <= date_latest_on_db]
df_trimmed.tail()

,trade_date,ticker,open_price,high_price,low_price,close_price,volume
3728351,2025-05-08,JXN,85.34,87.60,84.36,84.84,785700
3728352,2025-05-08,NEOG,5.97,6.12,5.86,5.99,6196300
3728353,2025-05-08,RHI,43.29,44.98,43.00,44.75,941300
3728354,2025-05-08,DVAX,9.63,9.90,9.28,9.69,3195300
3728355,2025-05-08,GDYN,13.56,14.25,13.40,14.16,579800


In [22]:
data = df_trimmed.to_dict(orient='records')

In [23]:
from src.utils import insert_data
from src.create_sql_tables import StockPrice

insert_data(data=data, table=StockPrice)